# Data Warehouse Iqvia QA - Member Enrollment Monthly

Performing QA on member_enrollment_monthly table in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Table Information

This table contains enrollment information on a monthly level. Depending on the data source, this information can be easily extracted in a monthly level.

Data Sources:

* Optum Zip/Optum DoD: Enrollment information not on monthly level. Enrollment dates have a begin date and an end  date which may be longer than a month. Enrollment tables are **mbr_enroll** and **mbr_co_enroll**
* Truven: Enrollment table , **t**, contains monthly level enrollment data
* Medicaid: Enrollment tables (**enrl**, **chip_uth**, **htw_enrl**) are in month level usually identified by **elig_month/elig_date** column
* Medicare: Enrollment table (**mbsf_abcd_summary**) are in yearly level, to get monthly enrollment, you need to look at the **mdcr_status_code_** columns
* Iqvia: enroll2 is the monthly level enrollment table, and enroll_synth contains demographic data of all members enrolled

Ideally we should have counts of enrollment tables from raw sources. These counts are included with the rest of the raw data tables counts for the given data sources.

* Optum Zip: **qa_reporting.optum_zip_counts**
* Optum Dod: **qa_reporting.optum_dod_counts**
* Medicaid: **qa_reporting.mdcd_enrollment_counts_[cy/fy]**
* Truven: **qa_reporting.truven_counts**
* Medicare: **qa_reporting.medicare_national_counts** and **qa_reporting.medicare_texas_counts**

## Row Counts and Enrollment Counts

In [6]:
query = ''' drop table if exists qa_reporting.dw_iqva_mbr_enrl_monthly;
create table qa_reporting.dw_iqva_mbr_enrl_monthly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    src_row_count int,
    row_count_diff int,
    row_count_diff_percentage float,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [7]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_iqva_mbr_enrl_monthly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        current_date
  from dw_staging.iqva_member_enrollment_monthly
 group by 1,2,3;
      '''

      cursor.execute(query)

In [17]:
with connection.cursor() as cursor:
      query = '''
update qa_reporting.dw_iqva_mbr_enrl_monthly a
set src_row_count = b.row_count,
      row_count_diff = a.dw_row_count - b.row_count,
      row_count_diff_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.pat_count,
      src_mbr_count = b.pat_count,
      mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
      mbr_count_percentage = 100. * abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.iqvia_counts b
where data_source = 'iqva'
and a.calendar_year = b.year
and a.table_src ||'2' = b.table_name
;
      '''

      cursor.execute(query)

After inserting the counts from the dw_staging schema, let's see if there are any years where the counts do not match with the raw tables.

In [18]:
query = '''
select * 
from qa_reporting.dw_iqva_mbr_enrl_monthly
order by calendar_year
;'''
member_monthly_df = pd.read_sql(query, con=connection)
member_monthly_df


C:\Users\iperez9\.conda\envs\psycog_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,iqva,2006,enroll,22334793,284739058,-262404265,855.408139,2208186,2208186,30675914,-28467728,92.801564,2024-01-16
1,iqva,2007,enroll,24317399,346468836,-322151437,925.170023,2325360,2325360,34820782,-32495422,93.321919,2024-01-16
2,iqva,2008,enroll,25658724,373065024,-347406300,932.863709,2437422,2437422,37240842,-34803420,93.454976,2024-01-16
3,iqva,2009,enroll,26430475,359279585,-332849110,945.367121,2488359,2488359,35208450,-32720091,92.932495,2024-01-16
4,iqva,2010,enroll,27896995,312564976,-284667981,906.984075,2640561,2640561,31386216,-28745655,91.586877,2024-01-16
5,iqva,2011,enroll,30086276,304518538,-274432262,882.749298,2815844,2815844,31088358,-28272514,90.942449,2024-01-16
6,iqva,2012,enroll,32001076,269545623,-237544547,841.719738,2978541,2978541,28221335,-25242794,89.445783,2024-01-16
7,iqva,2013,enroll,34650531,247873630,-213223099,833.574377,3218901,3218901,25579373,-22360472,87.416029,2024-01-16
8,iqva,2014,enroll,40865145,258722696,-217857551,763.239909,3999484,3999484,28543784,-24544300,85.988249,2024-01-16
9,iqva,2015,enroll,47058179,250504863,-203446684,771.189966,4431943,4431943,26380878,-21948935,83.200169,2024-01-16


In [6]:
member_monthly_df[(member_monthly_df['row_count_diff_percentage'] > 1.) | (member_monthly_df['mbr_count_percentage'] > 1.)]

,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated


If **member_monthly_df** does not have any rows, it means that all of the rows from the raw tables are in this enrollment table at a monthly level.

## Gender Count

Now that we have verified that most if not all of the rows from the raw tables, ccaet and mdcrt, have been added to the member_enrollment_monthly table, we will check that the counts for other columns such as gender have been correctly added to the DW table.

In this case we won't seperate the counts by source table, just by calendar year.

In [21]:
query = '''with iqva_gen_cd as (
    select year, a.pat_id, der_sex
    from iqvia.enroll2 a
    join iqvia.enroll_synth b
    on a.pat_id = b.pat_id
),
iqva_gen as (
    select year, der_sex, count(*) gender_count
    from iqva_gen_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.iqva_member_enrollment_monthly
    group by 1,2
)
select a.year, a.gender_cd, a.gender_count as dw_gender_count, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_count_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_count_diff_percentage
from iqva_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.der_sex;
'''
 
df = pd.read_sql(query,  con=connection)
df.sort_values(['year', 'gender_cd'])

,year,gender_cd,dw_gender_count,src_gender_count,gender_count_diff,gender_count_diff_percentage
15,2006,F,11761338,11761338,0,0.0
21,2006,M,10573018,10573018,0,0.0
24,2006,U,437,437,0,0.0
11,2007,F,12770861,12770861,0,0.0
39,2007,M,11546113,11546113,0,0.0
32,2007,U,425,425,0,0.0
3,2008,F,13459267,13459267,0,0.0
18,2008,M,12199036,12199036,0,0.0
16,2008,U,421,421,0,0.0
36,2009,F,13879840,13879840,0,0.0


## Plan Type Counts

In [22]:
# Including enrollments where the plantyp column is NULL. Treating it as if unknown.
query = '''with iqva_enroll as (
    select year, pat_id, plan_type
    from iqvia.enroll2 a
    left join reference_tables.ref_plan_type c
  	on c.data_source  = 'iqva' 
    and c.plan_type_src = a.prd_type
),
iqva_plans as (          
    select year, case when plan_type is null then 'U' else plan_type end as plan_type, count(*) plan_count
    from iqva_enroll a
    group by 1,2
),
dw_plans as (
    select year, case when plan_type is null then 'U' else plan_type end as plan_type,
            count(*) plan_count
    from dw_staging.iqva_member_enrollment_monthly
    group by 1,2
)
select a.year, a.plan_type, a.plan_count as dw_plan_count, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_count_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_count_diff_percentage
from iqva_plans b
full outer join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_count_df = pd.read_sql(query,  con=connection)
plan_count_df.sort_values(['year', 'plan_type'])

C:\Users\iperez9\.conda\envs\psycog_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
1,2006.0,CDHP,72808.0,715632,-642824.0,89.826056
2,2006.0,FFS,33.0,15492469,-15492436.0,99.999787
0,2006.0,HMO,5742437.0,81070648,-75328211.0,92.916750
4,2006.0,POS,497451.0,24654013,-24156562.0,97.982272
5,2006.0,PPO,16021669.0,160385054,-144363385.0,90.010497
...,...,...,...,...,...,...
113,NaN,None,NaN,6990,NaN,NaN
114,NaN,None,NaN,103850,NaN,NaN
115,NaN,None,NaN,103259,NaN,NaN
116,NaN,None,NaN,30552,NaN,NaN


In [23]:
plan_count_df[plan_count_df['plan_count_diff_percentage'] > 1.0]

,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
0,2006.0,HMO,5742437.0,81070648,-75328211.0,92.916750
1,2006.0,CDHP,72808.0,715632,-642824.0,89.826056
2,2006.0,FFS,33.0,15492469,-15492436.0,99.999787
3,2006.0,UNK,395.0,2390690,-2390295.0,99.983478
4,2006.0,POS,497451.0,24654013,-24156562.0,97.982272
...,...,...,...,...,...,...
73,2019.0,POS,4552517.0,5362142,-809625.0,15.098910
74,2019.0,PPO,59931448.0,76371942,-16440494.0,21.526877
75,2019.0,CDHP,10603796.0,13418514,-2814718.0,20.976376
76,2019.0,HMO,24872215.0,31374966,-6502751.0,20.725922


In [24]:
yearly_plan_count_df = plan_count_df.groupby('year')['dw_plan_count', 'src_plan_count'].sum()
yearly_plan_count_df['plan_count_diff'] = yearly_plan_count_df['dw_plan_count'] - yearly_plan_count_df['src_plan_count']
yearly_plan_count_df['plan_count_diff_percentage'] = 100.* abs(yearly_plan_count_df['plan_count_diff'] / yearly_plan_count_df['src_plan_count'])
yearly_plan_count_df

C:\Users\iperez9\AppData\Local\Temp\80\ipykernel_10072\850223352.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  yearly_plan_count_df = plan_count_df.groupby('year')['dw_plan_count', 'src_plan_count'].sum()


,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
year,,,,
2006.0,22334793.0,284708506,-262373713.0,92.155207
2007.0,24317399.0,346277769,-321960370.0,92.977488
2008.0,25658724.0,372871223,-347212499.0,93.118610
2009.0,26430475.0,359187614,-332757139.0,92.641596
2010.0,27896995.0,312461126,-284564131.0,91.071851
2011.0,30086276.0,304274880,-274188604.0,90.112139
2012.0,32001076.0,269202428,-237201352.0,88.112635
2013.0,34650531.0,247464114,-212813583.0,85.997755
2014.0,40865145.0,256098225,-215233080.0,84.043175
